In [7]:
import torch
import os

In [8]:
from roboflow import Roboflow
rf = Roboflow(api_key="BALoMBSgbpYsTlG6z7xc")
project = rf.workspace("yoloweek12").project("deteksi_sawit_week13")
version = project.version(1)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to deteksi_sawit_week13-1 in yolov5pytorch:: 100%|██████████| 109/109 [00:00<00:00, 4247.53it/s]


In [9]:
!python train.py --data /content/yolov5/deteksi_sawit_week13-1/data.yaml --cfg yolov5s.yaml --batch-size 16 --name Dataset --epochs 100 --img 640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-11-18 06:29:37.749424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 06:29:37.773762: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 06:29:37.780915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to regi

In [11]:
import torch
import rasterio
from shapely.geometry import Point, mapping
import fiona
from fiona.crs import from_epsg

In [14]:
model_path = "/content/yolov5/runs/train/Dataset/weights/best.pt"
raster_path = "/content/data/Raster_Sawit_AOI.tif"
gsd = 0.05
output_shp_path = '/content/hasil/hasil2.shp'

In [15]:
def predict_yolo_raster(model_path, raster_path, gsd, output_shp_path):
  model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
  with rasterio.open(raster_path) as src:
    transform = src.transform
    crs = src.crs
    img = src.read([1, 2, 3])
    img = img.transpose(1, 2, 0)
    results = model(img)
    boxes = results.xyxy[0].numpy()
    schema = {
      'geometry': 'Point',
      'properties': {
        'class_id': 'int',
        'confidence': 'float'
      }
    }
  with fiona.open(output_shp_path, 'w', 'ESRI Shapefile', schema=schema, crs=from_epsg(crs.to_epsg())) as shp:
    for box in boxes:
      x_min, y_min, x_max, y_max, confidence, class_id = box

      center_x = transform[2] + (x_min + x_max) / 2 * gsd
      center_y = transform[5] - (y_min + y_max) / 2 * gsd

      point = Point(center_x, center_y)

      shp.write({
        'geometry': mapping(point),
        'properties': {
          'class_id': int(class_id),
          'confidence': float(confidence)
        }
      })
      print(f"Shapefile saved at: {output_shp_path}")

In [18]:
predict_yolo_raster(model_path, raster_path, gsd, output_shp_path)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-18 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil/hasil2.shp
Shapefile saved at: /content/hasil

This function will be removed in version 2.0. Please use CRS.from_epsg() instead.


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

def calculate_metrics(ground_truth_shp, predicted_shp, distance_tolerance=5):
    # Load shapefile ground truth dan prediksi
    ground_truth = gpd.read_file(ground_truth_shp)
    predicted = gpd.read_file(predicted_shp)

    # Pastikan CRS (Coordinate Reference System) sama
    if ground_truth.crs != predicted.crs:
        predicted = predicted.to_crs(ground_truth.crs)

    # Inisialisasi matched sets
    matched_predictions = set()
    matched_ground_truth = set()

    # Loop over ground truth points untuk mencocokkan dengan prediksi
    for idx_gt, gt_point in enumerate(ground_truth.geometry):
        for idx_pred, pred_point in enumerate(predicted.geometry):
            if idx_pred in matched_predictions:
                continue  # Prediksi yang sudah dipasangkan diabaikan

            # Periksa apakah jarak prediksi cukup dekat dengan ground truth
            if gt_point.distance(pred_point) < distance_tolerance:
                matched_ground_truth.add(idx_gt)
                matched_predictions.add(idx_pred)
                break  # Stop pencocokan untuk ground truth ini

    # Hitung TP, FP, FN
    true_positives = len(matched_ground_truth)
    false_positives = len(predicted) - len(matched_predictions)
    false_negatives = len(ground_truth) - len(matched_ground_truth)

    # Precision, Recall, F1-Score
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Hasil metrics
    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'true_positives': true_positives,
        'false_positives': false_positives,
        'false_negatives': false_negatives
    }

    return metrics


In [ ]:
ground_truth_shp = "/content/ground_truth/ground_truth.shp"
predicted_shp = "/content/hasil/hasil1.shp"

distance_tolerance = 1

metrics = calculate_metrics(ground_truth_shp, predicted_shp, distance_tolerance)

print("Precision:", metrics['precision'])
print("Recall:", metrics['recall'])
print("F1-Score:", metrics['f1_score'])
print("True Positives:", metrics['true_positives'])
print("False Positives:", metrics['false_positives'])
print("False Negatives:", metrics['false_negatives'])


Precision: 0.29491525423728815
Recall: 0.7631578947368421
F1-Score: 0.42542787286063577
True Positives: 87
False Positives: 208
False Negatives: 27


In [ ]:
import numpy as np
import rasterio

# Path ke file raster
raster_path = '/content/data/Raster_Sawit_AOI.tif'

# Buka file raster
with rasterio.open(raster_path) as src:
    # Baca semua 3 band (Red, Green, Blue)
    bands = src.read([1, 2, 3])  # Band 1=Red, Band 2=Green, Band 3=Blue

    # Band menjadi array 3D (band, row, col)
    red_band = bands[0]
    green_band = bands[1]
    blue_band = bands[2]

    # Menggabungkan ketiga band RGB menjadi array 3D (row, col, 3) untuk RGB
    rgb = np.stack((red_band, green_band, blue_band), axis=-1)

    # Mengonversi RGB menjadi integer 24-bit (R, G, B digabung menjadi satu nilai)
    combined_rgb = (red_band << 16) + (green_band << 8) + blue_band

    # Menampilkan nilai RGB yang digabungkan pada pixel tertentu (misalnya, baris 100, kolom 200)
    row, col = 100, 200
    pixel_combined_rgb = combined_rgb[row, col]
    print(f"Nilai RGB yang digabungkan pada baris {row}, kolom {col}: {pixel_combined_rgb}")

    # Jika ingin mendapatkan semua nilai RGB yang digabungkan untuk seluruh gambar
    print("Dimensi gambar combined RGB:", combined_rgb.shape)  # Dimensi: (row, col)
    print("Nilai RGB yang digabungkan pada seluruh gambar:\n", combined_rgb)


Nilai RGB yang digabungkan pada baris 100, kolom 200: 16
Dimensi gambar combined RGB: (1919, 1919)
Nilai RGB yang digabungkan pada seluruh gambar:
 [[ 43  38  36 ...  69  68  75]
 [ 40  38  36 ...  75  70  75]
 [ 38  38  36 ... 100 100  75]
 ...
 [170 141 137 ...  40  49  49]
 [170 170 117 ...  23  49  73]
 [137 141 106 ...  75  75  73]]


In [ ]:
import numpy as np
import rasterio

# Path ke file raster
raster_path = '/content/data/Raster_Sawit_AOI.tif'

# Buka file raster
with rasterio.open(raster_path) as src:
    # Baca semua 3 band (Red, Green, Blue)
    bands = src.read([1, 2, 3])  # Band 1=Red, Band 2=Green, Band 3=Blue

    # Band menjadi array 3D (band, row, col)
    red_band = bands[0]
    green_band = bands[1]
    blue_band = bands[2]

    # Menggabungkan ketiga band RGB menjadi array 3D (row, col, 3) untuk RGB
    rgb = np.stack((red_band, green_band, blue_band), axis=-1)

    # Mengonversi RGB menjadi integer 24-bit (R, G, B digabung menjadi satu nilai)
    combined_rgb = (red_band << 16) + (green_band << 8) + blue_band

    # Fungsi untuk klasifikasi warna berdasarkan RGB
    def classify_color(R, G, B):
        if R > 150 and G < 100 and B < 100:
            return 'Merah'
        elif R < 100 and G > 150 and B < 100:
            return 'Hijau'
        elif R < 100 and G < 100 and B > 150:
            return 'Biru'
        elif R > 150 and G > 150 and B < 100:
            return 'Kuning'
        elif R > 100 and G > 100 and B > 100:
            return 'Abu-abu'
        elif R == 0 and G == 0 and B == 0:
            return 'Hitam'
        elif R == 255 and G == 255 and B == 255:
            return 'Putih'
        else:
            return 'Warna lain'

    # Menampilkan nilai RGB yang digabungkan pada pixel baris 1, kolom 1
    row, col = 1, 1  # Baris 1, Kolom 1
    pixel_combined_rgb = combined_rgb[row, col]

    # Mengonversi nilai RGB kembali dari nilai 24-bit integer
    red_value = (pixel_combined_rgb >> 16) & 0xFF
    green_value = (pixel_combined_rgb >> 8) & 0xFF
    blue_value = pixel_combined_rgb & 0xFF

    # Klasifikasi warna untuk pixel pada baris 1, kolom 1
    color_class = classify_color(red_value, green_value, blue_value)

    print(f"Nilai RGB yang digabungkan pada baris {row}, kolom {col}: {pixel_combined_rgb}")
    print(f"Nilai RGB pada pixel tersebut: ({red_value}, {green_value}, {blue_value})")
    print(f"Klasifikasi warna: {color_class}")


Nilai RGB yang digabungkan pada baris 1, kolom 1: 38
Nilai RGB pada pixel tersebut: (0, 0, 38)
Klasifikasi warna: Warna lain


In [ ]:
import numpy as np
import rasterio

# Path ke file raster
raster_path = '/content/data/Raster_Sawit_AOI.tif'

# Buka file raster
with rasterio.open(raster_path) as src:
    # Baca semua 3 band (Red, Green, Blue)
    bands = src.read([1, 2, 3])  # Band 1=Red, Band 2=Green, Band 3=Blue

    # Band menjadi array 3D (band, row, col)
    red_band = bands[0]
    green_band = bands[1]
    blue_band = bands[2]

    # Menggabungkan ketiga band RGB menjadi array 3D (row, col, 3) untuk RGB
    rgb = np.stack((red_band, green_band, blue_band), axis=-1)

    # Menghitung grayscale dari setiap pixel
    grayscale = 0.2989 * red_band + 0.5870 * green_band + 0.1140 * blue_band

    # Menampilkan grayscale pada pixel tertentu (misalnya, baris 100, kolom 200)
    row, col = 100, 200
    pixel_grayscale = grayscale[row, col]
    print(f"Nilai grayscale pada baris {row}, kolom {col}: {pixel_grayscale}")

    # Jika ingin mendapatkan semua nilai grayscale untuk seluruh gambar
    print("Dimensi gambar grayscale:", grayscale.shape)  # Dimensi: (row, col)
    print("Nilai grayscale pada seluruh gambar:\n", grayscale)


Nilai grayscale pada baris 100, kolom 200: 31.0263
Dimensi gambar grayscale: (1919, 1919)
Nilai grayscale pada seluruh gambar:
 [[     74.877      61.872      51.046 ...      80.809      95.989      123.07]
 [     58.643      61.872      51.046 ...      123.07       110.9      123.07]
 [     61.872      61.872      51.046 ...      129.16      129.16      123.07]
 ...
 [     217.18      198.26      186.53 ...      122.97      102.73      102.73]
 [     202.76      202.76       169.5 ...      87.902      102.73      107.83]
 [     186.53      198.26      154.65 ...      123.07      123.07      107.83]]


In [ ]:
import rasterio

# Membuka file raster
with rasterio.open('/content/data/Raster_Sawit_AOI.tif') as src:
    # Membaca semua band (misalnya RGB - band 1, 2, 3)
    red_band = src.read(1)   # Band pertama untuk Red
    green_band = src.read(2) # Band kedua untuk Green
    blue_band = src.read(3)  # Band ketiga untuk Blue

    # Mendapatkan nilai pixel pada baris 1 dan kolom 1 (indeks mulai dari 0)
    row, col = 0, 0  # Baris pertama (index 0) dan kolom pertama (index 0)

    # Mendapatkan nilai RGB pada posisi (row, col)
    red_value = red_band[row, col]
    green_value = green_band[row, col]
    blue_value = blue_band[row, col]

    # Menampilkan nilai RGB
    print(f"RGB value at row 1, column 1: ({red_value}, {green_value}, {blue_value})")


RGB value at row 1, column 1: (77, 80, 43)


In [ ]:
import rasterio

# Membuka file raster
with rasterio.open('/content/data/Raster_Sawit_AOI.tif') as src:
  for band in range(1, src.count + 1):
    band_data = src.read(band)
    print(f"Band {band} - Dimensi: {band_data.shape}")


Band 1 - Dimensi: (1919, 1919)
Band 2 - Dimensi: (1919, 1919)
Band 3 - Dimensi: (1919, 1919)
Band 4 - Dimensi: (1919, 1919)


In [ ]:
import rasterio
import numpy as np

# Membuka file raster
with rasterio.open('/content/data/Raster_Sawit_AOI.tif') as src:
    # Mengecek jumlah band dalam file raster
    band_count = src.count
    print(f"Jumlah band: {band_count}")

    # Jika file raster memiliki band alpha (biasanya di band ke-4)
    if band_count >= 4:
        # Membaca band RGB
        red_band = src.read(1)   # Band pertama untuk Red
        green_band = src.read(2) # Band kedua untuk Green
        blue_band = src.read(3)  # Band ketiga untuk Blue
        alpha_band = src.read(4)  # Band ke-4 untuk Alpha

        # Menggabungkan RGB dan Alpha menjadi matriks RGBA
        # Membuat matriks kosong untuk RGBA (shape: [height, width, 4])
        rgba_matrix = np.stack((red_band, green_band, blue_band, alpha_band), axis=-1)

        # Menampilkan matriks RGBA pada pixel pertama
        print(f"RGBA value at row 1, column 1: {rgba_matrix[0, 0]}")

        # Mengonversi nilai RGBA ke kode warna heksadesimal
        def rgba_to_hex(r, g, b, a):
            # Mengonversi nilai RGB dan Alpha menjadi format #RRGGBBAA
            return f"#{r:02X}{g:02X}{b:02X}{a:02X}"

        # Membuat matriks kode warna heksadesimal (RGBA)
        hex_matrix = np.vectorize(rgba_to_hex)(rgba_matrix[..., 0], rgba_matrix[..., 1], rgba_matrix[..., 2], rgba_matrix[..., 3])

        # Menampilkan sebagian matriks kode warna heksadesimal (misalnya 5x5 pertama)
        print("Partial Hexadecimal RGBA matrix (5x5 pertama):")
        print(hex_matrix[:5, :5])  # Menampilkan 5x5 matriks pertama

    else:
        print("File raster tidak memiliki band alpha.")


Jumlah band: 4
RGBA value at row 1, column 1: [ 77  80  43 255]
Partial Hexadecimal RGBA matrix (5x5 pertama):
[['#4D502BFF' '#394526FF' '#333624FF' '#2D3015FF' '#2D3015FF']
 ['#453928FF' '#394526FF' '#333624FF' '#333624FF' '#2D3015FF']
 ['#394526FF' '#394526FF' '#333624FF' '#2D3015FF' '#2D3015FF']
 ['#394526FF' '#453928FF' '#333624FF' '#2D3015FF' '#2D3015FF']
 ['#394526FF' '#4D502BFF' '#2D3015FF' '#1C2721FF' '#2D3015FF']]
